In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [2]:
k = 5
rep = 100
r_s2_test = []
r_s5_test = []
r_s10_test = []
r_s10_l2_test = []

f_s2_test = []
f_s5_test = []
f_s10_test = []
f_s10_l2_test = []

percent_num = 0.1
out_num = [-15,15]

In [3]:
def m_distance(x1,x2,cov_m):
    return( np.dot(np.dot( (x1 - x2).T, np.linalg.inv(cov_m) ), (x1 - x2)) )

## s5

### sigma weighted

In [16]:
f_s5_test = []
r_s5_test = []

In [17]:
for label in range(rep):
    
    label = label + 1
    print(label)
    rountine = 'D:/研究生文件/科研任务/functional_data/sim_data/s5/' ###
    file_data = rountine + 'all_data_0801(5)_' + str(label) + '.csv' ###
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x = my_data[:,1:len(my_data[0,:])]
    n = len(my_x[:,0])
    x_dim = len(my_x[0,:])
    
    my_x[random.sample(range(n),int(n*percent_num) ),-5:] = np.random.choice(out_num,size = ( int(n*percent_num) ,5)) ### noise
    
    file_data = rountine + 'y_label_0801(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y = my_data[:,1] - 1 ##注意减一
    my_y = my_y.reshape((len(my_y),1))

    file_data = rountine + 'all_data_test_0801(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x_test = my_data[:,1:len(my_data[0,:])]
    n_test = len(my_x_test[:,0])
    x_dim = len(my_x_test[0,:])

    file_data = rountine + 'y_label_test_0801(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y_test = my_data[:,1] - 1 ##注意减一
    my_y_test = my_y_test.reshape((len(my_y),1))
    
    
    label_sum = np.zeros((k,x_dim))
    label_num = np.zeros(k)
    label_cov = np.zeros((k,x_dim,x_dim)) ###
    
    for i in range(n):
        i_label = my_y[i,0]
        label_num[i_label] += 1
        label_sum[i_label,:] += my_x[i,:]
    
    label_mean = np.zeros((k,x_dim))
    for i in range(k):
        label_mean[i,:] = label_sum[i,:]/label_num[i]
        label_cov[i,:] = np.cov( ((my_x[np.where(my_y[:,0] == i),:])[0,:]) , rowvar=False) ###
        
    
    correct_num = 0
    esti_mat = np.zeros(n)
    for i in range(n):
        esti = np.zeros(k)
        for j in range(k):
            esti[j] = m_distance(my_x[i,:],label_mean[j,:], label_cov[j,:]) ###
        esti_mat[i] = esti.argmin()
        if esti.argmin() == my_y[i]:
            correct_num += 1
    print(correct_num/n)
    

    correct_num = 0
    esti_mat_test = np.zeros(n_test)
    for i in range(n_test):
        esti = np.zeros(k)
        for j in range(k):
            esti[j] = m_distance(my_x_test[i,:],label_mean[j,:], label_cov[j,:]) ###

        esti_mat_test[i] = esti.argmin()
        if esti.argmin() == my_y_test[i]:
            correct_num += 1

    cf_mat = np.zeros((k,k))
    for i in range(n_test):
        cf_mat[int(my_y_test[i]),int(esti_mat_test[i])] += 1
        
    print(correct_num/n_test)
    print(cf_mat)
    r_s5_test.append(correct_num/n_test)
    f_s5_test.append(f1_score(my_y_test, esti_mat_test, average='macro'))

1
0.75
0.6266666666666667
[[37.  1.  0.  2.  7.]
 [ 1. 30. 12.  0.  0.]
 [ 2. 13. 74.  0.  0.]
 [36.  9.  0. 17.  0.]
 [23.  1.  5.  0. 30.]]
2
0.7466666666666667
0.6733333333333333
[[47.  1.  0.  4.  8.]
 [ 0. 36. 12.  2.  2.]
 [ 1. 14. 69.  0.  4.]
 [29.  6.  2. 27.  0.]
 [13.  0.  0.  0. 23.]]
3
0.8033333333333333
0.6666666666666666
[[32.  0.  0. 12.  7.]
 [ 1. 15. 14. 11.  0.]
 [ 5.  2. 92.  8.  8.]
 [19.  0.  1. 35.  0.]
 [12.  0.  0.  0. 26.]]
4
0.7366666666666667
0.6066666666666667
[[26.  0.  1. 12. 18.]
 [ 0. 30.  9.  2.  1.]
 [ 0. 16. 65.  0.  3.]
 [30.  5.  6. 29.  0.]
 [10.  0.  5.  0. 32.]]
5
0.7133333333333334
0.6066666666666667
[[34.  1.  2.  1.  5.]
 [ 2. 15. 30.  1.  1.]
 [ 2.  1. 99.  0.  1.]
 [36.  5.  6. 13.  0.]
 [22.  0.  2.  0. 21.]]
6
0.7433333333333333
0.6866666666666666
[[ 5.  3.  1. 30. 18.]
 [ 0. 28.  7.  4.  1.]
 [ 0. 13. 77.  0.  5.]
 [ 3.  6.  0. 45.  1.]
 [ 0.  0.  2.  0. 51.]]
7
0.76
0.7166666666666667
[[17.  3.  0. 26. 11.]
 [ 0. 33.  9.  1.  0.]
 [ 0. 

61
0.78
0.6033333333333334
[[33.  1.  1. 32.  0.]
 [ 1. 21. 16.  7.  0.]
 [ 0.  0. 77.  5.  0.]
 [ 6.  0.  1. 42.  0.]
 [32.  0. 16.  1.  8.]]
62
0.76
0.61
[[48.  0.  0.  8.  3.]
 [ 3. 11. 29.  6.  1.]
 [ 2.  0. 81.  3.  0.]
 [28.  0.  4. 26.  0.]
 [19.  0. 11.  0. 17.]]
63
0.7566666666666667
0.5933333333333334
[[30.  0.  3. 23.  1.]
 [ 0. 21. 14.  9.  1.]
 [ 0. 13. 73.  2.  1.]
 [26.  2.  1. 36.  0.]
 [19.  0.  7.  0. 18.]]
64
0.8
0.6666666666666666
[[31.  0.  1. 28.  2.]
 [ 0. 23. 15.  2.  0.]
 [ 3. 17. 68.  1.  2.]
 [12.  6.  0. 49.  0.]
 [ 9.  0.  2.  0. 29.]]
65
0.7533333333333333
0.6633333333333333
[[37.  2.  0.  2. 12.]
 [ 0. 23. 16.  1.  1.]
 [ 0.  6. 86.  1.  3.]
 [36.  8.  3. 17.  2.]
 [ 6.  0.  2.  0. 36.]]
66
0.7766666666666666
0.6933333333333334
[[49.  2.  0.  3.  7.]
 [ 1. 22. 18.  1.  0.]
 [ 0.  7. 74.  0.  4.]
 [23.  6.  1. 27.  0.]
 [ 9.  0. 10.  0. 36.]]
67
0.7166666666666667
0.6333333333333333
[[57.  0.  0.  6.  1.]
 [ 0. 14. 19. 11.  0.]
 [ 0.  6. 65.  1.  1.]
 [32.

In [18]:
print(r_s5_test)
print(np.mean(r_s5_test))
print(np.std(r_s5_test))

print(f_s5_test)
print(np.mean(f_s5_test))
print(np.std(f_s5_test))

[0.6266666666666667, 0.6733333333333333, 0.6666666666666666, 0.6066666666666667, 0.6066666666666667, 0.6866666666666666, 0.7166666666666667, 0.68, 0.6866666666666666, 0.6433333333333333, 0.7166666666666667, 0.6766666666666666, 0.6633333333333333, 0.67, 0.59, 0.6466666666666666, 0.6166666666666667, 0.6233333333333333, 0.6966666666666667, 0.7266666666666667, 0.6266666666666667, 0.6533333333333333, 0.72, 0.6266666666666667, 0.6866666666666666, 0.6233333333333333, 0.6766666666666666, 0.7333333333333333, 0.6833333333333333, 0.7433333333333333, 0.6466666666666666, 0.7733333333333333, 0.6366666666666667, 0.6633333333333333, 0.6666666666666666, 0.64, 0.6933333333333334, 0.6333333333333333, 0.65, 0.6233333333333333, 0.7066666666666667, 0.6866666666666666, 0.6333333333333333, 0.6233333333333333, 0.7233333333333334, 0.6333333333333333, 0.6166666666666667, 0.6566666666666666, 0.6166666666666667, 0.6733333333333333, 0.71, 0.6033333333333334, 0.6833333333333333, 0.6166666666666667, 0.66, 0.676666666

### SVM

In [12]:
f_s5_test = []
r_s5_test = []

In [13]:
import cvxpy as cp
k = 5
s = -1/(k-1)/10
C_num = 3
rep = 30

In [14]:
for label in range(rep):
    label = label + 1
    print(label)
    rountine = 'D:/研究生文件/科研任务/functional_data/sim_data/s5/'
    file_data = rountine + 'all_data_0801(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x = my_data[:,1:len(my_data[0,:])]
    n = len(my_x[:,0])
    x_dim = len(my_x[0,:])
    
    my_x[random.sample(range(n),int(n*percent_num) ),-5:] = np.random.choice(out_num,size = ( int(n*percent_num) ,5)) ### noise
    
    file_data = rountine + 'y_label_0801(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y = my_data[:,1] - 1 ##注意减一
    my_y = my_y.reshape((len(my_y),1))

    file_data = rountine + 'all_data_test_0801(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x_test = my_data[:,1:len(my_data[0,:])]
    n = len(my_x_test[:,0])
    x_dim = len(my_x_test[0,:])

    file_data = rountine + 'y_label_test_0801(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y_test = my_data[:,1] - 1 ##注意减一
    my_y_test = my_y_test.reshape((len(my_y),1))
    
    
    
    
    my_omega = np.random.randn(x_dim,k)
    my_b = np.random.randn(k)
    C_num = 0.05
    correct_num = 0
    iter_num = 15  ###总迭代次数

    train_cr = []
    test_cr = []
    test_f1 = [] ######

    for i in range(n):
        esti = np.dot(my_x[i,:], my_omega) + my_b
        if esti.argmax() == my_y[i]:
            correct_num += 1

    for iter in range(iter_num):
        try:
            my_beta = np.zeros((n,k))
            ##### beta
            for i in range(n):
                pred_iter = (np.dot(my_x[i,:], my_omega) + my_b)
                pred_copy = pred_iter.copy()
                pred_iter[int(my_y[i])] = -float('inf')
                j = pred_iter.argmax()

                if((pred_copy[int(my_y[i])] - pred_copy[j]) < s):
                    my_beta[i,j] = C_num

            ##### function1
            my_alpha = cp.Variable((n,k))
            obj_function = 0

            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j]) *  my_x[i,:]

                s1 += cp.sum_squares(s1_1 - s1_2)

            s2 = 0
            for i in range(n):
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s2 += my_alpha[i,j2]

            obj_function = 0.5 * s1 - s2

            ##### constrains1
            constrains = []
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2])

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j])

                constrains += [(s1_1 - s1_2) == 0]

            for i in range(n):
                s1 = 0
                for j in range(k):
                    if j != my_y[i]:
                        s1 += my_alpha[i,j]
                        constrains += [s1 >= 0]
                        constrains += [s1 <= C_num]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains += [my_alpha[i,j] >= 0]

            prob = cp.Problem(cp.Minimize(obj_function), constrains)
            prob.solve()

            ##### calculate omega
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                my_omega[:,j] = s1_1 - s1_2

            #print(my_omega)

            #####function2
            my_eta = cp.Variable(n)
            my_b = cp.Variable(k)

            obj_function_2 = 0
            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += my_beta[i,j2]

                    if my_y[i] != j:
                        s1_2 += my_beta[i,j2]

                s1 += (s1_1 - s1_2) * my_b[j]

            obj_function_2 = C_num * cp.sum(my_eta) + s1

            ##### constrains2

            constrains_2 = []
            f_sim = np.dot(my_x, my_omega)

            for i in range(n):
                constrains_2 += [my_eta[i] >= 0]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains_2 += [my_eta[i] >= (1 - (f_sim[i,int(my_y[i])] + my_b[int(my_y[i])]) + f_sim[i,j] + my_b[j])]

            constrains_2 += [cp.sum(my_b) == 0]

            prob_2 = cp.Problem(cp.Minimize(obj_function_2), constrains_2)

            prob_2.solve()
            my_b = my_b.value
            #print(my_b)

            ### train set correct-ratio ###

            correct_num = 0
            esti_mat = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x[i,:], my_omega) + my_b
                esti_mat[i] = esti.argmax()
                if esti.argmax() == my_y[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat = np.zeros((k,k))
            for i in range(n):
                cf_mat[int(my_y[i]),int(esti_mat[i])] += 1
            #print(cf_mat)


            train_cr.append(correct_num/n)


            ### test set correct-ratio ###
            correct_num = 0
            esti_mat_test = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x_test[i,:], my_omega) + my_b
                esti_mat_test[i] = esti.argmax()  ###
                if esti.argmax() == my_y_test[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat_test = np.zeros((k,k))
            for i in range(n):
                cf_mat_test[int(my_y_test[i]),int(esti_mat_test[i])] += 1 ###
            #print(cf_mat_test)

            test_cr.append(correct_num/n)
            test_f1.append(f1_score(my_y_test, esti_mat_test, average='macro')) ######
    
        except BaseException:
            print('can not calculate')
            if(len(test_cr) == 0):
                break
            print(train_cr)
            print(cf_mat)
            print(test_cr)
            print(cf_mat_test)
            #r_s5_test.append(max(test_cr))
            break
        
    if((iter == (iter_num - 1)) and (iter != 0)):
        print(train_cr)
        print(cf_mat)

        print(test_cr)
        print(cf_mat_test)
        r_s5_test.append(max(test_cr))
        f_s5_test.append(max(test_f1))

1


D:\anaconda3\envs\env-cvxpy\lib\site-packages\cvxpy\problems\problem.py:1334: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


[0.6333333333333333, 0.7033333333333334, 0.7366666666666667, 0.7566666666666667, 0.7566666666666667, 0.7633333333333333, 0.7666666666666667, 0.7633333333333333, 0.7666666666666667, 0.7533333333333333, 0.7566666666666667, 0.7533333333333333, 0.75, 0.7466666666666667, 0.75]
[[63.  4.  4.  7.  5.]
 [ 3. 36.  7.  2.  0.]
 [ 3.  2. 67.  3.  5.]
 [ 7.  5.  4. 33.  1.]
 [ 9.  0.  3.  1. 26.]]
[0.6266666666666667, 0.7033333333333334, 0.7333333333333333, 0.7633333333333333, 0.7933333333333333, 0.7866666666666666, 0.7866666666666666, 0.7833333333333333, 0.7866666666666666, 0.78, 0.77, 0.7633333333333333, 0.7633333333333333, 0.7633333333333333, 0.7633333333333333]
[[35.  1.  0.  9.  2.]
 [ 1. 33.  9.  0.  0.]
 [ 2.  8. 76.  0.  3.]
 [ 9.  6.  6. 38.  3.]
 [ 8.  0.  3.  1. 47.]]
2
[0.45, 0.6933333333333334, 0.7733333333333333, 0.7766666666666666, 0.7533333333333333, 0.7666666666666667, 0.7766666666666666, 0.7866666666666666, 0.78, 0.7866666666666666, 0.78, 0.7866666666666666, 0.78, 0.7866666666666

[0.5033333333333333, 0.5733333333333334, 0.6766666666666666, 0.7466666666666667, 0.7366666666666667, 0.7766666666666666, 0.7666666666666667, 0.7666666666666667, 0.7766666666666666, 0.7666666666666667, 0.7733333333333333, 0.79, 0.7466666666666667, 0.75, 0.7966666666666666]
[[46.  5.  0.  6.  4.]
 [ 1. 42.  3.  2.  1.]
 [ 2. 10. 65.  2.  6.]
 [ 6.  2.  2. 54.  3.]
 [ 2.  0.  2.  2. 32.]]
[0.45, 0.5666666666666667, 0.67, 0.7566666666666667, 0.76, 0.7866666666666666, 0.7933333333333333, 0.7766666666666666, 0.78, 0.7766666666666666, 0.7933333333333333, 0.8033333333333333, 0.76, 0.7633333333333333, 0.7866666666666666]
[[37.  3.  0.  6.  7.]
 [ 2. 36. 13.  1.  0.]
 [ 0.  9. 80.  2.  2.]
 [ 7.  0.  3. 38.  0.]
 [ 6.  0.  3.  0. 45.]]
15


D:\anaconda3\envs\env-cvxpy\lib\site-packages\cvxpy\problems\problem.py:1334: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


[0.25666666666666665, 0.39666666666666667, 0.5266666666666666, 0.5566666666666666, 0.6366666666666667, 0.7133333333333334, 0.72, 0.7233333333333334, 0.72, 0.7166666666666667, 0.7266666666666667, 0.7166666666666667, 0.72, 0.7233333333333334, 0.7233333333333334]
[[39.  4.  0.  9.  9.]
 [ 5. 37.  5.  2.  1.]
 [ 4.  8. 63.  6. 10.]
 [ 1.  4.  2. 39.  4.]
 [ 3.  4.  2.  0. 39.]]
[0.26, 0.41, 0.5266666666666666, 0.58, 0.7, 0.7633333333333333, 0.7633333333333333, 0.7633333333333333, 0.7666666666666667, 0.7633333333333333, 0.77, 0.77, 0.77, 0.7666666666666667, 0.77]
[[27.  3.  0. 12.  9.]
 [ 2. 40.  2.  0.  0.]
 [ 0.  9. 79. 12.  5.]
 [ 2.  6.  0. 45.  2.]
 [ 1.  0.  3.  1. 40.]]
16


D:\anaconda3\envs\env-cvxpy\lib\site-packages\cvxpy\problems\problem.py:1334: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


[0.39, 0.6666666666666666, 0.6833333333333333, 0.73, 0.7366666666666667, 0.7266666666666667, 0.7433333333333333, 0.7233333333333334, 0.7333333333333333, 0.7266666666666667, 0.73, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.73]
[[48.  3.  4.  7.  1.]
 [ 2. 16. 12.  0.  1.]
 [ 2.  1. 93.  0.  3.]
 [13.  3.  8. 26.  5.]
 [10.  1.  2.  3. 36.]]
[0.3933333333333333, 0.6466666666666666, 0.6933333333333334, 0.7166666666666667, 0.7266666666666667, 0.73, 0.74, 0.7166666666666667, 0.7233333333333334, 0.7066666666666667, 0.73, 0.7133333333333334, 0.73, 0.7266666666666667, 0.7133333333333334]
[[43.  1.  0. 11.  5.]
 [ 4. 31. 18.  0.  0.]
 [ 2.  1. 73.  0.  2.]
 [13.  4. 15. 31.  4.]
 [ 3.  0.  2.  1. 36.]]
17
[0.6933333333333334, 0.7566666666666667, 0.77, 0.7733333333333333, 0.79, 0.7833333333333333, 0.7766666666666666, 0.7933333333333333, 0.7866666666666666, 0.7866666666666666, 0.7866666666666666, 0.79, 0.7866666666666666, 0.7866666666666666, 0.7866666666666666]
[[35.  2.  2.  7

D:\anaconda3\envs\env-cvxpy\lib\site-packages\cvxpy\problems\problem.py:1334: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


[0.39, 0.59, 0.6333333333333333, 0.63, 0.6666666666666666, 0.6633333333333333, 0.7033333333333334, 0.7166666666666667, 0.7333333333333333, 0.6966666666666667, 0.7333333333333333, 0.73, 0.7266666666666667, 0.7433333333333333, 0.7133333333333334]
[[36.  4.  1.  6.  4.]
 [ 2. 32.  9.  1.  1.]
 [ 6.  4. 69. 11.  3.]
 [ 0.  2.  3. 48.  4.]
 [13.  4.  6.  2. 29.]]
[0.42333333333333334, 0.6833333333333333, 0.73, 0.6866666666666666, 0.76, 0.6966666666666667, 0.73, 0.7366666666666667, 0.7733333333333333, 0.7066666666666667, 0.76, 0.77, 0.7633333333333333, 0.78, 0.76]
[[40.  7.  0.  6.  7.]
 [ 0. 40. 21.  1.  0.]
 [ 0.  2. 67.  8.  3.]
 [ 0.  2.  1. 40.  6.]
 [ 8.  0.  0.  0. 41.]]
28
[0.5066666666666667, 0.6733333333333333, 0.6933333333333334, 0.7133333333333334, 0.7333333333333333, 0.73, 0.7466666666666667, 0.7633333333333333, 0.7733333333333333, 0.7766666666666666, 0.7866666666666666, 0.8, 0.7866666666666666, 0.8066666666666666, 0.78]
[[41.  4.  1.  7.  1.]
 [ 2. 34. 11.  2.  0.]
 [ 1.  4. 80

In [15]:
print(r_s5_test)
print(np.mean(r_s5_test))
print(np.std(r_s5_test))

print(f_s5_test)
print(np.mean(f_s5_test))
print(np.std(f_s5_test))

[0.7933333333333333, 0.8066666666666666, 0.8133333333333334, 0.7866666666666666, 0.73, 0.7933333333333333, 0.7333333333333333, 0.8466666666666667, 0.8, 0.7566666666666667, 0.8366666666666667, 0.7833333333333333, 0.7766666666666666, 0.8033333333333333, 0.77, 0.74, 0.8, 0.8066666666666666, 0.8166666666666667, 0.82, 0.71, 0.78, 0.7266666666666667, 0.8333333333333334, 0.83, 0.82, 0.78, 0.8433333333333334, 0.8066666666666666, 0.77]
0.7904444444444445
0.03561869394046743
[0.782007595035822, 0.7972084195997239, 0.7790207191886299, 0.7869898690040793, 0.6860132488703918, 0.765711739241151, 0.6731920510707525, 0.8416565617572053, 0.7882231808022562, 0.745972512574173, 0.820356278002877, 0.7567718897797782, 0.7578879754111952, 0.7982161252700466, 0.7578100550448773, 0.7343476417492678, 0.7973445015735207, 0.7977446362297695, 0.8059012076073232, 0.8086254200435576, 0.7101173323830745, 0.7714633694467948, 0.686011579978768, 0.8255508207732435, 0.8168387776134631, 0.8177039088214763, 0.772392943996

## s2

### sigma weighted

In [23]:
f_s2_test = []
r_s2_test = []

In [24]:
for label in range(rep):
    
    label = label + 1
    print(label)
    rountine = 'D:/研究生文件/科研任务/functional_data/sim_data/s2/' ###
    file_data = rountine + 'all_data_0801(2)_' + str(label) + '.csv' ###
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x = my_data[:,1:len(my_data[0,:])]
    n = len(my_x[:,0])
    x_dim = len(my_x[0,:])
    
    my_x[random.sample(range(n),int(n*percent_num) ),-2:] = np.random.choice(out_num,size = ( int(n*percent_num) ,2)) ### noise
    
    file_data = rountine + 'y_label_0801(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y = my_data[:,1] - 1 ##注意减一
    my_y = my_y.reshape((len(my_y),1))

    file_data = rountine + 'all_data_test_0801(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x_test = my_data[:,1:len(my_data[0,:])]
    n_test = len(my_x_test[:,0])
    x_dim = len(my_x_test[0,:])

    file_data = rountine + 'y_label_test_0801(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y_test = my_data[:,1] - 1 ##注意减一
    my_y_test = my_y_test.reshape((len(my_y),1))
    
    
    label_sum = np.zeros((k,x_dim))
    label_num = np.zeros(k)
    label_cov = np.zeros((k,x_dim,x_dim)) ###
    
    for i in range(n):
        i_label = my_y[i,0]
        label_num[i_label] += 1
        label_sum[i_label,:] += my_x[i,:]
    
    label_mean = np.zeros((k,x_dim))
    for i in range(k):
        label_mean[i,:] = label_sum[i,:]/label_num[i]
        label_cov[i,:] = np.cov( ((my_x[np.where(my_y[:,0] == i),:])[0,:]) , rowvar=False) ###
        
    
    correct_num = 0
    esti_mat = np.zeros(n)
    for i in range(n):
        esti = np.zeros(k)
        for j in range(k):
            esti[j] = m_distance(my_x[i,:],label_mean[j,:], label_cov[j,:]) ###
        esti_mat[i] = esti.argmin()
        if esti.argmin() == my_y[i]:
            correct_num += 1
    print(correct_num/n)
    

    correct_num = 0
    esti_mat_test = np.zeros(n_test)
    for i in range(n_test):
        esti = np.zeros(k)
        for j in range(k):
            esti[j] = m_distance(my_x_test[i,:],label_mean[j,:], label_cov[j,:]) ###

        esti_mat_test[i] = esti.argmin()
        if esti.argmin() == my_y_test[i]:
            correct_num += 1

    cf_mat = np.zeros((k,k))
    for i in range(n_test):
        cf_mat[int(my_y_test[i]),int(esti_mat_test[i])] += 1
        
    print(correct_num/n_test)
    print(cf_mat)
    r_s2_test.append(correct_num/n_test)
    f_s2_test.append(f1_score(my_y_test, esti_mat_test, average='macro'))

1
0.93
0.9233333333333333
[[35.  0.  0.  4.  2.]
 [ 0. 37.  2.  4.  0.]
 [ 0.  3. 96.  0.  0.]
 [ 1.  1.  0. 54.  0.]
 [ 5.  0.  1.  0. 55.]]
2
0.9533333333333334
0.9
[[34.  0.  0.  6.  2.]
 [ 0. 57.  0.  2.  0.]
 [ 0.  4. 62.  0.  0.]
 [ 2.  6.  0. 59.  0.]
 [ 5.  0.  3.  0. 58.]]
3
0.93
0.91
[[28.  0.  0.  3.  2.]
 [ 0. 53.  1.  0.  0.]
 [ 0.  4. 81.  0.  0.]
 [ 5.  8.  0. 46.  0.]
 [ 3.  0.  1.  0. 65.]]
4
0.93
0.9033333333333333
[[28.  0.  0.  7.  3.]
 [ 0. 42.  2.  2.  0.]
 [ 0.  5. 80.  0.  0.]
 [ 5.  1.  0. 59.  0.]
 [ 3.  0.  1.  0. 62.]]
5
0.9033333333333333
0.9
[[27.  0.  0. 10.  6.]
 [ 0. 34.  0.  6.  0.]
 [ 0.  0. 92.  0.  0.]
 [ 5.  1.  0. 54.  1.]
 [ 0.  0.  1.  0. 63.]]
6
0.9133333333333333
0.9033333333333333
[[41.  0.  0.  5.  5.]
 [ 0. 49.  3.  0.  0.]
 [ 0.  4. 73.  0.  0.]
 [ 2.  3.  0. 56.  0.]
 [ 4.  0.  3.  0. 52.]]
7
0.9166666666666666
0.8866666666666667
[[25.  0.  0.  8.  7.]
 [ 0. 50.  2.  3.  0.]
 [ 0.  3. 77.  0.  0.]
 [ 5.  1.  0. 62.  0.]
 [ 2.  0.  3.  0. 

In [25]:
print(r_s2_test)
print(np.mean(r_s2_test))
print(np.std(r_s2_test))

print(f_s2_test)
print(np.mean(f_s2_test))
print(np.std(f_s2_test))

[0.9233333333333333, 0.9, 0.91, 0.9033333333333333, 0.9, 0.9033333333333333, 0.8866666666666667, 0.8833333333333333, 0.9066666666666666, 0.8966666666666666, 0.8933333333333333, 0.9133333333333333, 0.86, 0.8966666666666666, 0.8666666666666667, 0.8866666666666667, 0.8533333333333334, 0.9033333333333333, 0.8866666666666667, 0.8966666666666666, 0.88, 0.8733333333333333, 0.9133333333333333, 0.92, 0.9133333333333333, 0.88, 0.9466666666666667, 0.91, 0.91, 0.8766666666666667]
0.8964444444444444
0.01956817774466783
[0.9103531028706721, 0.8943671668504234, 0.8948740848996604, 0.8873075322046541, 0.8771974499279956, 0.8989248096411142, 0.8676954538614037, 0.8720036735098834, 0.8816668498331948, 0.8823986773965293, 0.885840324550002, 0.8948077475971215, 0.8431713374041738, 0.861946441870626, 0.8559323253678434, 0.8772619623636221, 0.8407568263333307, 0.8994495317958642, 0.8811350144687887, 0.8811581883072442, 0.8669720075976224, 0.8060124846079055, 0.9038213747037277, 0.9121586236098379, 0.8922652

### SVM

In [16]:
r_s2_test = []
f_s2_test = []

In [17]:
import cvxpy as cp
k = 5
s = -1/(k-1)/10
C_num = 3
rep = 30

In [18]:
for label in range(rep):
    label = label + 1
    print(label)
    rountine = 'D:/研究生文件/科研任务/functional_data/sim_data/s2/'
    file_data = rountine + 'all_data_0801(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x = my_data[:,1:len(my_data[0,:])]
    n = len(my_x[:,0])
    x_dim = len(my_x[0,:])
    
    my_x[random.sample(range(n),int(n*percent_num) ),-2:] = np.random.choice(out_num,size = ( int(n*percent_num) ,2)) ### noise
    
    file_data = rountine + 'y_label_0801(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y = my_data[:,1] - 1 ##注意减一
    my_y = my_y.reshape((len(my_y),1))

    file_data = rountine + 'all_data_test_0801(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x_test = my_data[:,1:len(my_data[0,:])]
    n = len(my_x_test[:,0])
    x_dim = len(my_x_test[0,:])

    file_data = rountine + 'y_label_test_0801(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y_test = my_data[:,1] - 1 ##注意减一
    my_y_test = my_y_test.reshape((len(my_y),1))
    
    
    
    
    my_omega = np.random.randn(x_dim,k)
    my_b = np.random.randn(k)
    C_num = 0.05
    correct_num = 0
    iter_num = 15  ###总迭代次数

    train_cr = []
    test_cr = []
    test_f1 = [] ######

    for i in range(n):
        esti = np.dot(my_x[i,:], my_omega) + my_b
        if esti.argmax() == my_y[i]:
            correct_num += 1

    for iter in range(iter_num):
        try:
            my_beta = np.zeros((n,k))
            ##### beta
            for i in range(n):
                pred_iter = (np.dot(my_x[i,:], my_omega) + my_b)
                pred_copy = pred_iter.copy()
                pred_iter[int(my_y[i])] = -float('inf')
                j = pred_iter.argmax()

                if((pred_copy[int(my_y[i])] - pred_copy[j]) < s):
                    my_beta[i,j] = C_num

            ##### function1
            my_alpha = cp.Variable((n,k))
            obj_function = 0

            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j]) *  my_x[i,:]

                s1 += cp.sum_squares(s1_1 - s1_2)

            s2 = 0
            for i in range(n):
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s2 += my_alpha[i,j2]

            obj_function = 0.5 * s1 - s2

            ##### constrains1
            constrains = []
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2])

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j])

                constrains += [(s1_1 - s1_2) == 0]

            for i in range(n):
                s1 = 0
                for j in range(k):
                    if j != my_y[i]:
                        s1 += my_alpha[i,j]
                        constrains += [s1 >= 0]
                        constrains += [s1 <= C_num]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains += [my_alpha[i,j] >= 0]

            prob = cp.Problem(cp.Minimize(obj_function), constrains)
            prob.solve()

            ##### calculate omega
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                my_omega[:,j] = s1_1 - s1_2

            #print(my_omega)

            #####function2
            my_eta = cp.Variable(n)
            my_b = cp.Variable(k)

            obj_function_2 = 0
            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += my_beta[i,j2]

                    if my_y[i] != j:
                        s1_2 += my_beta[i,j2]

                s1 += (s1_1 - s1_2) * my_b[j]

            obj_function_2 = C_num * cp.sum(my_eta) + s1

            ##### constrains2

            constrains_2 = []
            f_sim = np.dot(my_x, my_omega)

            for i in range(n):
                constrains_2 += [my_eta[i] >= 0]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains_2 += [my_eta[i] >= (1 - (f_sim[i,int(my_y[i])] + my_b[int(my_y[i])]) + f_sim[i,j] + my_b[j])]

            constrains_2 += [cp.sum(my_b) == 0]

            prob_2 = cp.Problem(cp.Minimize(obj_function_2), constrains_2)

            prob_2.solve()
            my_b = my_b.value
            #print(my_b)

            ### train set correct-ratio ###

            correct_num = 0
            esti_mat = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x[i,:], my_omega) + my_b
                esti_mat[i] = esti.argmax()
                if esti.argmax() == my_y[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat = np.zeros((k,k))
            for i in range(n):
                cf_mat[int(my_y[i]),int(esti_mat[i])] += 1
            #print(cf_mat)


            train_cr.append(correct_num/n)


            ### test set correct-ratio ###
            correct_num = 0
            esti_mat_test = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x_test[i,:], my_omega) + my_b
                esti_mat_test[i] = esti.argmax() ####
                if esti.argmax() == my_y_test[i]:  
                    correct_num += 1
            #print(correct_num/n)

            cf_mat_test = np.zeros((k,k))
            for i in range(n):
                cf_mat_test[int(my_y_test[i]),int(esti_mat_test[i])] += 1
            #print(cf_mat_test)

            test_cr.append(correct_num/n)
            test_f1.append(f1_score(my_y_test, esti_mat_test, average='macro')) ######

    
        except BaseException:
            print('can not calculate')
            if(len(test_cr) == 0):
                break
            print(train_cr)
            print(cf_mat)
            print(test_cr)
            print(cf_mat_test)
            #r_s2_test.append(max(test_cr))
            break
        
    if((iter == (iter_num - 1)) and (iter != 0)):
        print(train_cr)
        print(cf_mat)

        print(test_cr)
        print(cf_mat_test)
        r_s2_test.append(max(test_cr))
        f_s2_test.append(max(test_f1))

1
[0.5933333333333334, 0.7, 0.6866666666666666, 0.6933333333333334, 0.6933333333333334, 0.6933333333333334, 0.6933333333333334, 0.6933333333333334, 0.6933333333333334, 0.6933333333333334, 0.6933333333333334, 0.6933333333333334, 0.6933333333333334, 0.6933333333333334, 0.6933333333333334]
[[ 0.  1.  3. 30. 14.]
 [ 0. 39.  1.  1.  0.]
 [ 0. 15. 66.  2.  3.]
 [ 0. 12.  4. 39.  4.]
 [ 0.  0.  1.  1. 64.]]
[0.6333333333333333, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76, 0.76]
[[ 0.  0.  0. 29. 12.]
 [ 0. 43.  0.  0.  0.]
 [ 0. 14. 85.  0.  0.]
 [ 0. 17.  0. 39.  0.]
 [ 0.  0.  0.  0. 61.]]
2
[0.63, 0.7933333333333333, 0.77, 0.77, 0.77, 0.77, 0.77, 0.77, 0.77, 0.77, 0.77, 0.77, 0.77, 0.77, 0.77]
[[ 0.  2.  0. 20. 13.]
 [ 0. 33.  1.  3.  3.]
 [ 0.  6. 80.  3.  1.]
 [ 0. 13.  0. 61.  1.]
 [ 0.  0.  2.  1. 57.]]
[0.62, 0.79, 0.7666666666666667, 0.7666666666666667, 0.7666666666666667, 0.7666666666666667, 0.7666666666666667, 0.7666666666666667, 0.7666666666666667,

D:\anaconda3\envs\env-cvxpy\lib\site-packages\cvxpy\problems\problem.py:1334: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
D:\anaconda3\envs\env-cvxpy\lib\site-packages\cvxpy\problems\problem.py:1334: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


[0.2733333333333333, 0.32, 0.32, 0.5066666666666667, 0.6266666666666667, 0.6566666666666666, 0.6966666666666667, 0.73, 0.7233333333333334, 0.7233333333333334, 0.7233333333333334, 0.7233333333333334, 0.7233333333333334, 0.7233333333333334, 0.7233333333333334]
[[ 0.  0.  1. 23. 11.]
 [ 0. 59.  2.  0.  4.]
 [ 0. 14. 68.  4.  2.]
 [ 0. 13.  3. 41.  1.]
 [ 0.  4.  1.  0. 49.]]
[0.23, 0.26666666666666666, 0.26666666666666666, 0.47, 0.62, 0.67, 0.75, 0.79, 0.7966666666666666, 0.7966666666666666, 0.7966666666666666, 0.7966666666666666, 0.7966666666666666, 0.7966666666666666, 0.7966666666666666]
[[ 0.  0.  0. 17. 15.]
 [ 0. 49.  0.  0.  0.]
 [ 0. 14. 70.  0.  0.]
 [ 0. 15.  0. 60.  0.]
 [ 0.  0.  0.  0. 60.]]
10
[0.6066666666666667, 0.7133333333333334, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7]
[[ 0.  1.  1. 24. 11.]
 [ 0. 46.  1.  2.  0.]
 [ 0. 19. 66.  1.  2.]
 [ 0. 19.  1. 37.  0.]
 [ 0.  4.  3.  1. 61.]]
[0.6533333333333333, 0.7766666666666666, 0.7466666666666667, 0.74

can not calculate
23
[0.5333333333333333, 0.6066666666666667, 0.7133333333333334, 0.7033333333333334, 0.71, 0.7333333333333333, 0.73, 0.73, 0.73, 0.73, 0.73, 0.73, 0.73, 0.73, 0.73]
[[ 0.  1.  0. 30.  8.]
 [ 0. 37.  1.  0.  2.]
 [ 0. 11. 64.  1.  4.]
 [ 0. 12.  3. 55.  4.]
 [ 0.  2.  1.  1. 63.]]
[0.5666666666666667, 0.6633333333333333, 0.7333333333333333, 0.75, 0.7466666666666667, 0.7533333333333333, 0.75, 0.7433333333333333, 0.7433333333333333, 0.7433333333333333, 0.7433333333333333, 0.7433333333333333, 0.7433333333333333, 0.7433333333333333, 0.7433333333333333]
[[ 0.  0.  0. 35. 16.]
 [ 0. 55.  0.  0.  0.]
 [ 0. 12. 65.  0.  0.]
 [ 0. 14.  0. 46.  0.]
 [ 0.  0.  0.  0. 57.]]
24
[0.6366666666666667, 0.7, 0.7266666666666667, 0.73, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333]
[[ 2.  2.  5. 23.  6.]
 [ 0. 49.  0.  

In [19]:
print(r_s2_test)
print(np.mean(r_s2_test))
print(np.std(r_s2_test))

print(f_s2_test)
print(np.mean(f_s2_test))
print(np.std(f_s2_test))

[0.76, 0.79, 0.7066666666666667, 0.78, 0.7566666666666667, 0.6833333333333333, 0.77, 0.7966666666666666, 0.7766666666666666, 0.75, 0.8166666666666667, 0.87, 0.7866666666666666, 0.77, 0.7166666666666667, 0.76, 0.6633333333333333, 0.7666666666666667, 0.7533333333333333, 0.7666666666666667, 0.83, 0.6866666666666666, 0.8033333333333333, 0.7233333333333334, 0.7166666666666667]
0.7599999999999999
0.046780812780930214
[0.6544044270014187, 0.684838036549779, 0.5344690131386514, 0.6644872778751868, 0.6633116883116883, 0.4917073333896681, 0.6638940936719221, 0.6794461002244703, 0.6693315594258991, 0.6589799440610211, 0.7378176985328486, 0.8261127370415486, 0.6763494321257599, 0.66119523855537, 0.5712642312642313, 0.6770780271018253, 0.4969831109275654, 0.6581528340080972, 0.705424329365216, 0.6775969286188264, 0.6886396609926021, 0.57104793472718, 0.6857205587411537, 0.5136448990105058, 0.5037251230324181]
0.640624888707794
0.08052990876185147
